# HRRR Dashboard
The High Resolution Rapid Refresh (HRRR, pronouned "her") is the highest resolution (2.5km) weather forecast for the entire Continental USA.  Here we investigate the gridded HRRR forecast data products from [Unidata's THREDDS server](http://thredds.ucar.edu) and visualizing the data using the [pyviz](pyviz.org) tools. 

In [ ]:
import xarray as xr

In [ ]:
ds = xr.open_dataset('http://thredds.ucar.edu/thredds/dodsC/grib/NCEP/HRRR/CONUS_2p5km/Best')

Drop the "reftime" coordinate variables, as hvplot has trouble with this

In [ ]:
ds = ds.drop([coord for coord in ds.coords if 'reftime' in coord])

Find all the data variables that depend on time (and are not time `bounds`)

In [ ]:
time_vars = []
for var in ds.data_vars:
    if len(ds[var].dims) > 0:
        if 'time' in ds[var].dims[0] and not 'bounds' in var:
            time_vars.append(var)

Import the [pyviz](http://pyviz.org) tools we need

In [ ]:
from cartopy import crs as ccrs
import hvplot.xarray
import holoviews as hv
from geoviews import tile_sources as gvts
import panel as pn

Create widget for variable selection

In [ ]:
var_select = pn.widgets.Select(name='HRRR Variables:', options=time_vars, 
                               value='Temperature_height_above_ground')

Create widget for basemap selection

In [ ]:
ds['LambertConformal_Projection']

In [ ]:
base_map_select = pn.widgets.Select(name='Basemap:', options=gvts.tile_sources, value=gvts.OSM)                                                    

Create a color mesh plot in Lambert Conformal coordinates with `hvplot` and `cartopy`

In [ ]:
globe = ccrs.Globe(ellipse='sphere', semimajor_axis=ds.LambertConformal_Projection.earth_radius)
lat0 = ds.LambertConformal_Projection.latitude_of_projection_origin
lon0 = ds.LambertConformal_Projection.longitude_of_central_meridian
lat1 = ds.LambertConformal_Projection.standard_parallel

Convert coordinates from kilometers to meters so that projection will work

In [ ]:
ds['x'].values = ds['x'].values*1000.
ds['y'].values = ds['y'].values*1000.

In [ ]:
crs = ccrs.LambertConformal(central_latitude=lat0, central_longitude=lon0,
                            standard_parallels=(lat0,lat1), globe=globe)

The `plot` function below creates the `hvplot` panel layout object.  We specify a basemap, pick the `quadmesh` plot type for the selected variable, and indicate we want to `rasterize` the plot so that we can render massive meshes in the browser. We also specify the `groupby` parameter as the list of dimensions that remains after we remove Y and X: `ds[var].dims[:-2]`, which automatically handles variables with either dimensions [T, Y, X] or [T, Z, Y, X].  We also specify which `bokeh` controls we want to be active by default:  the `wheel_zoom` and `pan` controls.

We also change the default slider to a selection widget for the `time` dimension so that specific times are easy to select.  See https://stackoverflow.com/a/54912917/2005869

In [ ]:
def plot(var=None, base_map=None):
    base_map = base_map or base_map_select.value
    var = var or var_select.value

    mesh = ds[var].hvplot.quadmesh(x='x', y='y', rasterize=True, crs=crs, title=var,
                                   width=600, height=400, 
                                   groupby=list(ds[var].dims[:-2]), cmap='jet')

    overlay = (base_map * mesh.opts(alpha=0.7)).opts(active_tools=['wheel_zoom', 'pan'])
    widgets = {ds[var].dims[0]: pn.widgets.Select}
    return pn.holoviews.HoloViews(overlay, widgets=widgets).layout

In [ ]:
def on_var_select(event):
    var = event.obj.value
    col[-1] = plot(var=var)

In [ ]:
def on_base_map_select(event):
    base_map = event.obj.value
    col[-1] = plot(base_map=base_map)

In [ ]:
var_select.param.watch(on_var_select, parameter_names=['value']);
base_map_select.param.watch(on_base_map_select, parameter_names=['value']);

In [ ]:
col = pn.Column(var_select, base_map_select, plot(var_select.value))

We use `.servable()` below not only to display the panel object, but to makes the panel servable outside the notebook via:  `panel serve HRRR_Dashboard.ipynb`

In [ ]:
col.servable()